<a href="https://colab.research.google.com/github/dresvyankina/dz/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22More_about_SQL_solved_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/19e_jJVbRqbD6eaCr31zn7xY6fSP2NYIG?usp=sharing

## План занятия

- Повторим как создавать БД при помощи SQLalchemy
- Разберем новые операторы в SQL
- Разберем подробнее ORM (объектно-реляционное отображение) в SQLalchemy

(задание из презентации)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
prices = np.random.permutation(np.arange(10))
prices

array([9, 2, 3, 0, 7, 5, 8, 1, 6, 4])

In [ ]:
k = 3
for i in range(len(prices)):
    n_leq = 0
    for j in range(len(prices)):
        if prices[j] >= prices[i]:
            n_leq += 1
    if n_leq == k:
        print(prices[i])
        break

7


# SQLAlchemy. Recap

In [ ]:
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In [ ]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)


In [ ]:
Base = declarative_base()

class Person(Base):
    __tablename__= 'person'
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    country = Column(String(250))
    state = Column(String(250))
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250))
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)

engine = create_engine('sqlite:///sqlalchemy_example.db')

connection = engine.connect()

Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:

DBSession = sessionmaker(bind=engine)
session = DBSession()

new_person = Person(name='Nawin')
session.add(new_person)
session.commit()

new_address = Address(post_code='48103',country = 'USA', state = 'WA', street_name ='West Huron Street',street_number ='1', person=new_person)
session.add(new_address)
session.commit()

In [ ]:
engine.table_names()

['address', 'person']

In [ ]:
from sqlalchemy import select 

query = select([Person])
display_results(query)

,id,name
0,1,Nawin
1,2,Nawin
2,3,Moshe
3,4,Lisa
4,5,Nika
5,6,Max
6,7,Dan
7,8,Alex
8,9,Nawin
9,10,Max


In [ ]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,USA,WA,West Huron Street,1,48103,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,Russia,Moscow,Pervomay Street,42,141700,4
4,5,USA,CA,Green Street,104,655689,5
5,6,USA,WA,West Huron Street,1,48103,9
6,7,USA,WA,West Huron Street,1,48103,13


Добавим несколько людей с разными адресами

In [ ]:
from sqlalchemy import exists
def record_exists(session, name):
    return session.query(exists().where(Person.name == name)).scalar()

In [ ]:
record_exists(session, 'Nawin')

True

In [ ]:
record_exists(session, 'Lisa')

True

In [ ]:
names = ['Moshe', 'Lisa', 'Nika']
adress_info = [('141700', 'Russia', 'Moscow', 'Pervomay Street', '42'), #Moshe adress
               ('141700', 'Russia', 'Moscow', 'Pervomay Street', '42'), #Lisa adress
               ('655689', 'USA', 'CA', 'Green Street', '104')] #Nika adress

In [ ]:
for i, name, adress in zip(range(len(names)), names, adress_info):
    if not record_exists(session, name):
        new_person = Person(name=name)
        new_address = Address(country=adress[1],state=adress[2], street_number=adress[4], post_code=adress[0],street_name =adress[3], person=new_person)
        session.add(new_person)
        session.add(new_address)
    else:
        print("Already exists, skipping...")

session.commit()

Already exists, skipping...
Already exists, skipping...
Already exists, skipping...


Проверим теперь таблицу

In [ ]:
query = select([Person.id, Person.name])
display_results(query)

,id,name
0,1,Nawin
1,2,Nawin
2,3,Moshe
3,4,Lisa
4,5,Nika
5,6,Max
6,7,Dan
7,8,Alex
8,9,Nawin
9,10,Max


In [ ]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,USA,WA,West Huron Street,1,48103,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,Russia,Moscow,Pervomay Street,42,141700,4
4,5,USA,CA,Green Street,104,655689,5
5,6,USA,WA,West Huron Street,1,48103,9
6,7,USA,WA,West Huron Street,1,48103,13


Другой способ добавления с использованием **add_all()**

In [ ]:
session.add_all([Person(name='Max'),
                 Person(name='Dan'),
                 Person(name='Alex')])
session.commit()

In [ ]:
query = select([Person.id, Person.name])
display_results(query)

,id,name
0,1,Nawin
1,2,Nawin
2,3,Moshe
3,4,Lisa
4,5,Nika
5,6,Max
6,7,Dan
7,8,Alex
8,9,Nawin
9,10,Max


In [ ]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,USA,WA,West Huron Street,1,48103,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,Russia,Moscow,Pervomay Street,42,141700,4
4,5,USA,CA,Green Street,104,655689,5
5,6,USA,WA,West Huron Street,1,48103,9
6,7,USA,WA,West Huron Street,1,48103,13


# Синтаксис 'чистых' SQL запросов. Повторение + новый материал

## Create

In [ ]:
engine.table_names()

['address', 'person']

In [ ]:
connection.execute('''
CREATE TABLE friends (
    id_ INT UNSIGNED AUTO_INCREMENT,
    friendName VARCHAR(50),
    phone VARCHAR(50),
    addressLine1 VARCHAR(50),
    addressLine2 VARCHAR(50),
    city VARCHAR(50),
    state VARCHAR(50),
    postalCode VARCHAR(50),
    country VARCHAR(50),
    PRIMARY KEY(id_)
);
''')

In [ ]:
engine.table_names()

['address', 'friends', 'person']

In [ ]:
result = connection.execute("select * from friends")
for row in result:
    print("Friend info", row)

In [ ]:
pd.read_sql('SELECT * from friends;', connection)

,id_,friendName,phone,addressLine1,addressLine2,city,state,postalCode,country


In [ ]:
pd.read_sql('SELECT * from person;', connection)

,id,name
0,1,Nawin
1,2,Nawin
2,3,Moshe
3,4,Lisa
4,5,Nika
5,6,Max
6,7,Dan
7,8,Alex
8,9,Nawin
9,10,Max


In [ ]:

result = connection.execute(
'''SELECT
    country,
    state,
    post_code
FROM 
    address
WHERE 
    country = 'USA' AND 
    state = 'CA';
''')

for row in result:
    print("adress info", row)

adress info ('USA', 'CA', '655689')


## Insert into

In [ ]:
connection.execute(
"""
INSERT INTO friends (
    id_,
    country,
    state,
    postalCode
)
VALUES (1, "UK", "London", 5678)
"""
)

In [ ]:
pd.read_sql('SELECT * from friends;', connection)

,id_,friendName,phone,addressLine1,addressLine2,city,state,postalCode,country
0,1,None,None,None,None,None,London,5678,UK


## Update

In [ ]:
connection.execute(
'''
UPDATE friends 

SET
    friendName = "Mike"
WHERE 
    country = "UK"
''')

In [ ]:
pd.read_sql('SELECT * from friends;', connection)

,id_,friendName,phone,addressLine1,addressLine2,city,state,postalCode,country
0,1,Mike,None,None,None,None,London,5678,UK


## Insert into select

In [ ]:
connection.execute(
'''
INSERT INTO friends (
    country,
    state,
    postalCode
)
SELECT
    country,
    state,
    post_code
FROM 
    address
WHERE 
    country = 'USA' AND 
    state = 'CA';
''')

In [ ]:
pd.read_sql('SELECT * from friends;', connection)

,id_,friendName,phone,addressLine1,addressLine2,city,state,postalCode,country
0,1.0,Mike,None,None,None,None,London,5678,UK
1,NaN,None,None,None,None,None,CA,655689,USA


## Drop

In [ ]:
connection.execute('DROP TABLE friends;')

In [ ]:
engine.table_names()

['address', 'person']

## Create view

CREATE VIEW создаст представление по заданному вами условию

In [ ]:
connection.execute('''
CREATE VIEW test_view AS
    SELECT 
        street_name, 
        street_number
    FROM
        address
    ORDER BY street_number;
''')

In [ ]:
engine.table_names()

['address', 'person']

Но в pandas таблицы и представления разделяют одно пространство имен 

In [ ]:
pd.read_sql('SELECT * from test_view;', connection)

,street_name,street_number
0,West Huron Street,1
1,West Huron Street,1
2,West Huron Street,1
3,West Huron Street,1
4,Green Street,104
5,Pervomay Street,42
6,Pervomay Street,42


In [ ]:
connection.execute('''
drop view test_view;
''')

## Case

Выражение MySQL CASE - это структура потока управления, которая позволяет добавлять логику if-else к запросу. Вообще говоря, вы можете использовать выражение CASE в любом месте, которое допускает допустимое выражение, например, предложения SELECT, WHERE и ORDER BY.

In [ ]:
pd.read_sql('SELECT * from address;', connection)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,USA,WA,West Huron Street,1,48103,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,Russia,Moscow,Pervomay Street,42,141700,4
4,5,USA,CA,Green Street,104,655689,5
5,6,USA,WA,West Huron Street,1,48103,9
6,7,USA,WA,West Huron Street,1,48103,13


In [ ]:
pd.read_sql('''
SELECT 
    
    country `Страна`,
    CASE country
		WHEN "Russia" THEN 'Живет в России'
        WHEN "USA" THEN 'Живет в США'
        ELSE 'Живет в другой стране'
	end `Страна проживания`
FROM
    (SELECT 
		country
	FROM
		address
	INNER JOIN person
		ON  address.person_id = person.id) as cte
ORDER BY country;
''', connection)

,Страна,Страна проживания
0,Russia,Живет в России
1,Russia,Живет в России
2,USA,Живет в США
3,USA,Живет в США
4,USA,Живет в США
5,USA,Живет в США
6,USA,Живет в США


ЗАДАНИЕ

Добавить еще один столбец - Имя человека

In [ ]:
# TODO

In [ ]:
pd.read_sql('''
SELECT 
    name 'Имя',
    country 'Страна',
    CASE country
		WHEN "Russia" THEN 'Живет в России'
        WHEN "USA" THEN 'Живет в США'
        ELSE 'Живет в другой стране'
	end 'Страна проживания'
FROM
    (SELECT 
		country,
        name
	FROM
		address
	INNER JOIN person
		USING (id))
ORDER BY country;
''', connection)

,Имя,Страна,Страна проживания
0,Moshe,Russia,Живет в России
1,Lisa,Russia,Живет в России
2,Nawin,USA,Живет в США
3,Nawin,USA,Живет в США
4,Nika,USA,Живет в США
5,Max,USA,Живет в США
6,Dan,USA,Живет в США


Вопрос: Почему в данном случае плохо использовать следующий синтаксис "JOIN ... USING (id)" ?

Как это исправить?

In [ ]:
#TODO

In [ ]:
pd.read_sql('''
SELECT 
    name 'Имя',
    country 'Страна',
    CASE state
		WHEN "Russia" THEN 'Живет в России'
        WHEN "USA" THEN 'Живет в США'
        ELSE 'Живет в другой стране'
	end 'Страна проживания'
FROM
    (SELECT 
		country,
        name
	FROM
		address
	INNER JOIN person
		ON  address.person_id = person.id)
ORDER BY country;
''', connection)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: ignored

In [ ]:
# TODO

ДОП ЗАДАНИЕ**

Если человек живет в Москве, то вывести его имя, улицу и почтовый код

In [ ]:
pd.read_sql('''
SELECT 
    name 'Имя',
    street_number 'Улица',
    post_code 'Почтовый код'
    CASE state
		WHEN "Moscow" THEN 'Живет в Москве'
        ELSE 'Живет в другом городе'
	end 'Страна проживания'
FROM
    (SELECT 
		    name,
        street_number,
        post_code
	FROM
		address
	INNER JOIN person
		ON  address.person_id = person.id)
ORDER BY state;
''', connection)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: ignored

In [ ]:
#попытка решения 2 задачи

In [ ]:
INSERT INTO new (id, Количество_строк)
SELECT
    id AS id, count(*) AS Количество_строк
FROM tablica;